In [ ]:
# This is supposed to make sure if you edit .py files
# colab will pick up on it
%load_ext autoreload
%autoreload 2

In [ ]:
!rm -rf PROJECT
!git clone https://github.com/iggy2k/CSC490-Project-Winter-2025.git PROJECT

Cloning into 'PROJECT'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 220 (delta 79), reused 175 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (220/220), 14.95 MiB | 32.86 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [ ]:
!ls PROJECT/PIGEON

bot	   dataset_creation  get_auxiliary_data.sh  preprocessing  runs
config.py  env.yml	     LICENSE.txt	    README.md	   saved_models
data	   evaluation	     models		    run.py	   training


In [ ]:
import sys
sys.path.insert(1, 'PROJECT/PIGEON')
print(sys.path)

['/usr/local/lib/python3.11/site-packages', 'PROJECT/PIGEON', '/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython']


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

✨🍰✨ Everything looks OK!


In [ ]:
!mamba env update -n base -f "PROJECT/PIGEON/env.yml"

Channels:
 - pytorch
 - huggingface
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: / - failed

SpecsConfigurationConflictError: Requested specs conflict with configured specs.
  requested specs: 
    - folium=0.13.0
    - huggingface_hub=0.10.1
    - ipywidgets=7.6.5
    - jinja2=3.0.3
    - jupyter=1.0.0
    - markupsafe=2.1.1
    - matplotlib-base=3.5.3
    - matplotlib-inline=0.1.2
    - nbconvert=6.4.4
    - nbformat=5.3.0
    - notebook=6.4.11
    - numpy=1.23.1
    - pandas=1.4.2
    - pip=22.2.2
    - pynvml=11.4.1
    - python=3.10
    - pytorch=1.12.1
    - scikit-learn=1.1.1
    - scipy=1.9.1
    - tensorboard=2.9.0
    - tensorboard-data-server=0.6.1
    - tensorboard-plugin-wit=1.8.1
    - torchvision=0.13.1
    - tornado=6.1
    - tqdm=4.64.1
  pinned specs: 
    - cuda-version=12
    - python=3.11
    - python_abi=3.11[build=*cp311*]
Use 'conda config --show-sources' to look for 'pinned_specs' and 'track_features'
configuration parameters.  Pinned 

In [ ]:
!python --version

Python 3.11.11


In [ ]:
!pip install -q global_land_mask

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="osv5m/osv5m", local_dir="datasets/osv5m", allow_patterns=[
    'images/train/00.zip',
    'images/test/00.zip',
    '*.csv'
    ], repo_type='dataset')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

'/content/datasets/osv5m'

In [ ]:
import csv
import numpy as np
import pandas as pd
from shapely.geometry import Point

import geopandas as gpd
from geopandas import GeoDataFrame

import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import zipfile
from time import sleep

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from torchvision import datasets, transforms

from PIL import Image
import pandas as pd

from tqdm.notebook import trange, tqdm

# from global_land_mask import globe

In [ ]:
for root, dirs, files in os.walk("datasets/osv5m"):
    for file in files:
        if file.endswith(".zip"):
            with zipfile.ZipFile(os.path.join(root, file), 'r') as zip_ref:
                for member in tqdm(zip_ref.infolist(), desc=f'Extracting {os.path.join(root, file)}'):
                  zip_ref.extract(member, root)
            os.remove(os.path.join(root, file))

Extracting datasets/osv5m/images/train/00.zip:   0%|          | 0/50001 [00:00<?, ?it/s]

Extracting datasets/osv5m/images/test/00.zip:   0%|          | 0/50001 [00:00<?, ?it/s]

In [ ]:
# Optional (?). PIGEON uses full country names instead of alpha_2
!pip install -q pycountry

In [ ]:
import pycountry
class ImageCoordinateDataset(Dataset):
    def __init__(self, csv_file, image_dirs, train: bool, transform=None, detach=False):
        self.data = []
        self.skipped = []
        self.files = []
        self.images = []
        self.detach = detach

        # Not sure if not storing all lat/long in memory actually affects
        # training time. But it should be a tradeoff between quick loading the dataset
        # versus losing a few miliseconds on every model call
        self.FAST_LOAD = True

        for image_dir in image_dirs:
          self.files.extend([f"{image_dir}/{f}" for f in listdir(image_dir) if isfile(join(image_dir, f))])
        self.csv = 'datasets/osv5m/train.csv' if train else 'datasets/osv5m/test.csv'

        print('Reading', self.csv)

        if self.FAST_LOAD:
          self.df = pd.read_csv(self.csv, usecols=['id', 'latitude', 'longitude', 'country'])
          # ex. DE -> Germany
          # Optional (?)
          self.df['country'] = self.df['country'].apply(lambda x: pycountry.countries.get(alpha_2=x))
        else:
          df = pd.read_csv(self.csv, usecols=['id', 'latitude', 'longitude', 'country'])
          for full_path in tqdm(self.files, total=len(self.files), desc='Processing files'):
              image_name = str(Path(full_path).stem)

              row = df[df['id'] == int(image_name)].iloc[0]

              lat = row['latitude']
              lon = row['longitude']

              # Remove mislaballed images (ocean pictures?)
              # if not globe.is_land(float(lat), float(lon)):
              #   self.skipped.append(image_name)
              #   continue
              self.data.append(np.array([str(full_path), float(lat), float(lon)]))

        self.transform = transform
        print(f'Dataset ready, {len(self.files)} files.')
        # print(f'Skipped {len(self.skipped)} non-land files.')


    def __len__(self):
        if self.FAST_LOAD:
          return len(self.files)
        else:
          return len(self.data)

    def __getitem__(self, idx):
        if self.FAST_LOAD:
            img_path = self.files[idx]
            row = self.df[self.df['id'] == int(str(Path(img_path).stem))].iloc[0]
            lat = row['latitude']
            lon = row['longitude']
            coordinates = (float(lat), float(lon))
        else:
            img_path = self.data[idx][0]
            coordinates = (float(self.data[idx][1]), float(self.data[idx][2]))

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.detach:
          return image, torch.tensor(coordinates, dtype=torch.float32).detach()
        return image, torch.tensor(coordinates, dtype=torch.float32)


In [ ]:
# https://www.geeksforgeeks.org/how-to-normalize-images-in-pytorch/
# The mean and std of ImageNet are: mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].
mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32)
std = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32)
normalize = transforms.Normalize(mean.tolist(), std.tolist())

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

In [ ]:
BATCH_SIZE = 32

In [ ]:
train_dataset = ImageCoordinateDataset(csv_file='datasets/osv5m/train.csv', image_dirs=[
                                      '/content/datasets/osv5m/images/train/00',
                                      ],\
                                 train=True,
                                 transform=transform)
val_dataset = ImageCoordinateDataset(csv_file='datasets/osv5m/train.csv', image_dirs=[
                                      '/content/datasets/osv5m/images/test/00',
                                      ],\
                                 train=False,
                                 transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

Reading datasets/osv5m/train.csv
Dataset ready, 50000 files.
Reading datasets/osv5m/test.csv
Dataset ready, 50000 files.


In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('using cuda')
else:
  device = torch.device("cpu")
  print('using cpu')
torch.cuda.empty_cache()

using cpu


In [ ]:
CLIP_MODEL = 'openai/clip-vit-large-patch14-336'

In [ ]:
from transformers import AutoModelForImageClassification, CLIPVisionModel
embed_model = CLIPVisionModel.from_pretrained(CLIP_MODEL)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [ ]:
GEOCELL_PATH_YFCC = 'data/geocells_yfcc.csv'
METADATA_PATH_YFCC = 'data/data_yfcc_augmented_non_contaminated.csv'
PRETRAIN_METADATA_PATH_YFCC = 'data/data_yfcc_augmented_non_contaminated.csv'

In [ ]:
!mkdir -p data/geocells/
# Political boundaries (admin0 = country)
![ ! -f data/geocells/admin_2.geojson ] && wget --show-progress -O data/geocells/admin_2.geojson -q https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM2.geojson
![ ! -f data/geocells/admin_1.geojson ] && wget --show-progress -O data/geocells/admin_1.geojson -q https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM1.geojson
![ ! -f data/geocells/countries.geojson ] && wget --show-progress -O data/geocells/countries.geojson -q https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM0.geojson

data/geocells/admin 100%[===================>] 524.95M   276MB/s    in 1.9s    
data/geocells/admin 100%[===================>] 343.82M   288MB/s    in 1.2s    
data/geocells/count 100%[===================>] 382.61M   291MB/s    in 1.3s    


In [ ]:
# TODO: pre-generate all this, add to the repo and add option to load instead
import sys
sys.path.insert(0,'PROJECT/PIGEON/dataset_creation/geocell')
sys.path.insert(0,'PROJECT/PIGEON/')
from geocell_creation import *

df = train_dataset.df
print(df.columns)
geocell_creator = GeocellCreator(df, 'data/geocells_yfcc.csv')
geocells = geocell_creator.generate()

Index(['id', 'latitude', 'longitude', 'country'], dtype='object')
Loading geographic boundaries ...
 ... loaded admin 2 boundaries.
 ... loaded admin 1 boundaries.
 ... loaded countries.


Fusing cells within countries:  57%|█████▋    | 111/196 [12:41<31:58, 22.57s/country]

In [ ]:
!pip list | grep geopandas

In [ ]:
# !python PROJECT/PIGEON/dataset_creation/geocell/geocell_creation.py

In [ ]:
import torchvision
from torchvision import datasets, transforms

import torch
import pandas as pd
from torch import nn, Tensor
from torch.nn.parameter import Parameter
from collections import namedtuple
from preprocessing import haversine_matrix, smooth_labels
from models.layers import PositionalEncoder
from models.utils import ModelOutput
from config import *

#TODO: move over more stuff from PIGEON

class GeoLocationModel(nn.Module):
    def __init__(self):
        super(GeoLocationModel, self).__init__()

        # Save variables
        self.base_model = embed_model

        resnet = torchvision.models.resnet50(pretrained=True).to(device)
        for i, param in enumerate(resnet.parameters()):
          if i < 4:
            param.requires_grad = False


        # Setup
        self._set_hidden_size()
        geocell_path = GEOCELL_PATH_YFCC if self.yfcc else GEOCELL_PATH
        self.lla_geocells = self.load_geocells(geocell_path)
        self.num_cells = self.lla_geocells.size(0)

        self.resnet = resnet
        num_feats = self.resnet.fc.in_features
        self.fc1 = nn.Linear(num_feats, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 2)



    def forward(self, x):
        features = self.resnet(x)

        x = self.fc1(features)
        x = self.relu(x)
        x = self.fc2(x)

        return x

    def load_geocells(self, path: str) -> Tensor:
        """Loads geocell centroids and converts them to ECEF format

        Args:
            path (str, optional): path to geocells. Defaults to GEOCELL_PATH.

        Returns:
            Tensor: ECEF geocell centroids
        """
        geo_df = pd.read_csv(path)
        lla_coords = torch.tensor(geo_df[['longitude', 'lat']].values)
        lla_geocells = nn.parameter.Parameter(data=lla_coords, requires_grad=False)
        return lla_geocells

    def _set_hidden_size(self):
        """
        Determines the hidden size of the model
        """
        if self.base_model is not None:
            try:
                self.hidden_size = self.base_model.config.hidden_size
                self.mode = 'transformer'

            except AttributeError:
                self.hidden_size = self.base_model.config.hidden_sizes[-1]
                self.mode = 'convnext'

model = GeoLocationModel()

# model = model.to(device)

In [ ]:
from transformers import CLIPProcessor, CLIPModel

In [ ]:
clip_model = CLIPModel.from_pretrained("laion/CLIP-ViT-L-14-laion2B-s32B-b82K")
clip_processor = CLIPProcessor.from_pretrained("laion/CLIP-ViT-L-14-laion2B-s32B-b82K")
clip_model.to(device)

In [ ]:
# def compute_clip(processor, model, x):
#     inputs = processor(images=x[0], return_tensors="pt", device=device).to(device)
#     features = model.get_image_features(**inputs)
#     features /= features.norm(dim=-1, keepdim=True)
#     features = features.cpu().detach().numpy()
#     for i in range(len(x[0])):
#         yield features[i].tolist()

In [ ]:
# import gc

# clip_model.cpu()
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# for images, coordinates in (tqdm(clip_dataloader, total=len(clip_dataloader), desc=f'Extracting')):
#   features = compute_clip(clip_processor, clip_model, images)
#   for feature in features:
#     print(feature)

In [ ]:
# https://github.com/gastruc/osv5m/blob/main/models/losses.py
class HaversineLoss(nn.Module):
    def __init__(self):
        super(HaversineLoss, self).__init__()

    def forward(self, x, y):
        """
        Args:
            x: torch.Tensor Bx2
            y: torch.Tensor Bx2
        Returns:
            torch.Tensor: Haversine loss between x and y: torch.Tensor([B])
        Note:
            Haversine distance doesn't contain the 2 * 6371 constant.
        """
        lhs = torch.sin((x[:, 0] - y[:, 0]) / 2) ** 2
        rhs = (
            torch.cos(x[:, 0])
            * torch.cos(y[:, 0])
            * torch.sin((x[:, 1] - y[:, 1]) / 2) ** 2
        )
        a = lhs + rhs
        return torch.arctan2(torch.sqrt(a), torch.sqrt(1 - a))

In [ ]:
from torch.optim.lr_scheduler import ExponentialLR

haversineLoss = HaversineLoss()
mseLoss = nn.MSELoss()
ceLoss = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = ExponentialLR(optimizer, gamma=0.9, verbose=True)

In [ ]:
!pip install -q livelossplot

In [ ]:
from livelossplot import PlotLosses

In [ ]:
from ipywidgets import Output
OUTPUT_CONTEXT = Output()
display(OUTPUT_CONTEXT)

In [ ]:
num_epochs = 5
avg_epoch_loss = []

plotlosses = PlotLosses(figsize=(10, 5))


for epoch in tqdm(range(num_epochs), total=num_epochs, desc='Epochs'):
  train_loss = []
  avg_loss = 0

  for images, coordinates in (pbar:= tqdm(train_dataloader, total=len(train_dataloader), desc=f'Epoch {epoch + 1} progress')):
    images, coordinates = images.cuda(), coordinates.cuda()

    optimizer.zero_grad()
    outputs = model(images)

    loss_haversine = haversineLoss(outputs, coordinates).mean(dim=-1)

    mse = mseLoss(outputs, coordinates)
    cross_entropy = ceLoss(outputs, coordinates)

    pbar.set_postfix_str(f'Loss: {loss_haversine:.5f}; \
                               \nOut: {outputs.mean(dim=0).tolist()}\
                               \nExpected: {coordinates.mean(dim=0).tolist()}')
    train_loss.append(loss_haversine.detach())

    with OUTPUT_CONTEXT:
      plotlosses.update({
          'Haversine': loss_haversine.item(),
          'MSE': mse.item(),
          'Cross Entropy': cross_entropy.item()
      })
      plotlosses.send()

    loss_haversine.backward()
    optimizer.step()

  scheduler.step()
  avg_loss = sum(train_loss) / len(train_dataloader)
  avg_epoch_loss.append(avg_loss)
  print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

In [ ]:
import matplotlib.pyplot as plt
avg_epoch_loss = torch.tensor(avg_epoch_loss, device =  'cpu')
plt.figure(figsize=(7, 5))
plt.plot(range(1, len(avg_epoch_loss) + 1), avg_epoch_loss, label='Loss', color='blue', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs Epoch')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
model.eval()

In [ ]:
avg_val_haversine_loss = 0.0
batch_haversine = []

with torch.no_grad():  # Disable gradient calculation for validation
    for images, coordinates in tqdm(val_dataloader, total=len(val_dataloader), desc=f'Validating'):
        # Forward pass
        images, coordinates = images.cuda(), coordinates.cuda()
        outputs = model(images)

        loss_haversine = haversineLoss(outputs, coordinates).mean(dim=-1)
        batch_haversine.append(loss_haversine)
        avg_val_haversine_loss += loss_haversine

# Calculate average loss and haversine
avg_val_haversine_loss /= len(val_dataloader)

print(f'Validation Haversine Loss: {avg_val_haversine_loss:.4f}')

In [ ]:
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter

# Back from gpu
batch_haversine = torch.tensor(batch_haversine, device =  'cpu')

# Remove outliers
batch_haversine = median_filter(batch_haversine, size=100)

plt.figure(figsize=(5, 5))
plt.subplot(1, 1, 1)
plt.plot(range(1, len(batch_haversine) + 1), batch_haversine, label='Validation haversine', color='green', marker='o')
plt.xlabel('Batch Number')
plt.ylabel('haversine')
plt.title('Validation haversine loss per Batch')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Step 5: Visualize some predictions (optional)
import matplotlib.pyplot as plt
import random

url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"

unnormalize = transforms.Normalize((-mean / std).tolist(), (1.0 / std).tolist())

def plot_predictions(model, dataloader, num_samples=5):
    model.eval()
    with torch.no_grad():
        for images, coordinates in dataloader:

            images = images.cuda()

            outputs = model(images)

            images = images.cpu()
            outputs = outputs.cpu()

            rand_index = random.sample(range(0, len(images) - 1), min(num_samples, len(images) - 1))

            for i in range(min(num_samples, len(images))):
                i = rand_index[i]

                pred_lat, pred_lon = outputs[i].cpu().numpy()
                true_lat, true_lon = coordinates[i].numpy()

                haver_err = haversineLoss(
                                      torch.tensor(np.array([[pred_lon, pred_lat]]), dtype=torch.float32).deg2rad(),
                                      torch.tensor(np.array([[true_lon, true_lat]]), dtype=torch.float32).deg2rad(),
                                      )
                # Display the image
                img = images[i]
                img = unnormalize(img).permute(1, 2, 0).numpy()
                img = np.clip(img, 0, 1)

                plt.imshow(img)
                plt.title(f'Pred: ({pred_lat:.4f}, {pred_lon:.4f})\nTrue: ({true_lat:.4f}, {true_lon:.4f})\n Haversine: {haver_err}')
                plt.axis('off')

                # World map for better understanding of how bad our prediction is
                geometry = [Point(pred_lon, pred_lat), Point(true_lon, true_lat)]
                geo_df = GeoDataFrame(geometry = geometry)
                world = gpd.read_file(url)
                geo_df.plot(ax=world.plot(color="lightgrey", figsize=(10, 6)), marker='x', c=['red', 'green'], markersize=50);

                plt.show()
            break

In [ ]:
plot_predictions(model, val_dataloader, num_samples=10)

# Test with other loss functions

code are GPT generated over multiple guiding prompts

In [ ]:
from torch.nn import L1Loss, SmoothL1Loss, CosineSimilarity, CrossEntropyLoss

l1Loss = L1Loss()
smoothL1Loss = SmoothL1Loss()
cosineSimilarity = CosineSimilarity(dim=1)
ceLoss = CrossEntropyLoss()

# Define all individual loss functions
loss_functions = {
    'Haversine': haversineLoss,
    'MSE': mseLoss,
    'L1': l1Loss,
    'Smooth L1': smoothL1Loss,
    'Cosine Similarity': lambda x, y: 1 - cosineSimilarity(x, y).mean(),
    'Cross Entropy': ceLoss
}

# Define different combinations of losses
loss_combinations = {
    'Haversine + MSE': lambda x, y: haversineLoss(x, y).mean() + 0.1 * mseLoss(x, y).mean(),
    'Haversine + L1': lambda x, y: haversineLoss(x, y).mean() + 0.1 * l1Loss(x, y).mean(),
    'Haversine + Smooth L1': lambda x, y: haversineLoss(x, y).mean() + 0.1 * smoothL1Loss(x, y).mean(),
    'Haversine + Cosine Similarity': lambda x, y: haversineLoss(x, y).mean() + (1 - cosineSimilarity(x, y).mean()),
    'MSE + L1': lambda x, y: mseLoss(x, y).mean() + 0.1 * l1Loss(x, y).mean()
}

# Combine the dictionaries for easier iteration
all_losses = {**loss_functions, **loss_combinations}

num_epochs = 5
results = {}

plotlosses = PlotLosses(figsize=(10, 5))

for epoch in tqdm(range(num_epochs), total=num_epochs, desc='Training'):
    train_loss = {name: [] for name in all_losses.keys()}

    # Rotate loss function every epoch
    chosen_loss_name = list(all_losses.keys())[epoch % len(all_losses)]
    chosen_loss_fn = all_losses[chosen_loss_name]
    print(f"\n🔄 Training with {chosen_loss_name} loss for this epoch...")

    for images, coordinates in tqdm(train_dataloader, total=len(train_dataloader), desc=f'Epoch {epoch + 1} progress'):
        images, coordinates = images.to(device), coordinates.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        # Compute all losses
        losses = {name: loss_fn(outputs, coordinates).mean() for name, loss_fn in all_losses.items()}

        # Normalize losses for fair comparison
        normalized_losses = {name: loss / loss.detach().abs().mean() for name, loss in losses.items()}

        # Choose the rotating loss function for backpropagation
        chosen_loss = losses[chosen_loss_name]

        chosen_loss.backward()
        optimizer.step()

        # Store losses for logging
        for name, loss in normalized_losses.items():
            train_loss[name].append(loss.item())

        # Visualization
        with OUTPUT_CONTEXT:
            plotlosses.update({name: loss.item() for name, loss in normalized_losses.items()})
            plotlosses.send()

    # Print average losses per epoch
    for name in all_losses.keys():
        avg_loss = sum(train_loss[name]) / len(train_dataloader)
        print(f'{name} Loss - Epoch [{epoch + 1}/{num_epochs}]: {avg_loss:.4f}')

    results[chosen_loss_name] = avg_loss
    scheduler.step()

print("\n🔍 Final Training Losses:")
for name, loss in results.items():
    print(f'{name}: {loss:.4f}')

# Validation step based on the last trained loss function
print("\n🧪 Evaluating on Validation Set (Last Trained Loss: {chosen_loss_name})...\n")
avg_val_loss = 0

with torch.no_grad():
    for images, coordinates in tqdm(val_dataloader, total=len(val_dataloader), desc=f'Validating {chosen_loss_name}'):
        images, coordinates = images.to(device), coordinates.to(device)
        outputs = model(images)

        loss = chosen_loss_fn(outputs, coordinates).mean()
        avg_val_loss += loss.item()

avg_val_loss /= len(val_dataloader)
print(f'✅ {chosen_loss_name} Validation Loss: {avg_val_loss:.4f}')
